In [66]:
# imports
from subprocess import check_output
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import plotly
import re
import nbformat
import time
import json
import scipy
import numpy as np
import plotly.graph_objects as go

In [7]:
# measure probability calculation efficiency
results_df = pd.DataFrame(columns = ['Processed Cells', 'Retries', 'Time', 'Examinations', 'Movements', 'Actions', 'Terrain'])
agents = ["Agent 6", "Agent 7", "Agent 8"]

for x in range(100):
  command = 'python main.py -d 50 -p 0.3'
  out = json.loads(str(check_output(command, cwd='/Users/naveenanyogeswaran/Desktop/School/probabilistic-sensing/src', shell=True)).lstrip("b'").rstrip("'").replace("\\n", ""))

  for i,a in enumerate(agents):
    results_df = results_df.append([{'Agent': i+6, 'Processed Cells': out[a]["processed"], 'Retries': out[a]["retries"], 'Time': out[a]["time"], 'Examinations': out[a]["examinations"], 'Movements': out[a]["movements"], 'Actions': out[a]["examinations"] + out[a]["movements"], 'Terrain': out[a]["terrain"]}], ignore_index=True)

results_df.head()

,Processed Cells,Retries,Time,Examinations,Movements,Actions,Terrain,Agent
0,7717,763,5.511110,518,1676,2194,2,6.0
1,61468,2116,13.096124,1393,4798,6191,2,7.0
2,18851,1759,9.591271,1262,3822,5084,2,8.0
3,258342,4333,15.145135,3504,26534,30038,3,6.0
4,198616,2798,13.179374,1868,7321,9189,3,7.0


In [2]:
results_df = pd.read_csv('misc/test_results_terrain3.csv', index_col=0)

In [3]:
results_df['Ratio'] = results_df['Movements'] / results_df['Examinations']
results_df.head()

,Processed Cells,Retries,Time,Examinations,Movements,Actions,Terrain,Agent,Ratio
0,7717,763,5.511110,518,1676,2194,2,6.0,3.235521
1,61468,2116,13.096124,1393,4798,6191,2,7.0,3.444365
2,18851,1759,9.591271,1262,3822,5084,2,8.0,3.028526
3,258342,4333,15.145135,3504,26534,30038,3,6.0,7.572489
4,198616,2798,13.179374,1868,7321,9189,3,7.0,3.919165


In [9]:
results_df.to_csv('misc/test_results_terrain3.csv')

In [65]:
results_df = results_df.convert_dtypes()
grouped_df = results_df.groupby(['Agent'])
avg_results_df = grouped_df.mean()
# avg_results_df = avg_results_df.reset_index()
# avg_results_df['Terrain'] = avg_results_df['Terrain'].map({2:'Flat', 3:'Hilly', 4:'Forest'})
avg_results_df = avg_results_df.reset_index()
avg_results_df

,Agent,Processed Cells,Retries,Time,Examinations,Movements,Actions,Terrain,Ratio
0,6,207891.79,4536.03,15.94875,3901.08,31501.81,35402.89,2.94,5.643324
1,7,135222.66,3608.12,14.416686,2876.68,18735.82,21612.5,2.94,4.384623
2,8,124936.07,2845.77,20.860659,3116.97,9656.66,12773.63,2.94,3.0598


In [72]:
table_df = avg_results_df.round(3)
table = go.Figure(data=[go.Table(
    header=dict(values=['Agent', 'Examinations', 'Movements', 'Ratio', 'Actions'],
                line_color='darkslategray',
                fill_color='lightskyblue',
                align='left'),
    cells=dict(values=[avg_results_df.Agent, avg_results_df.Examinations, avg_results_df.Movements, table_df.Ratio, avg_results_df.Actions],
               line_color='darkslategray',
               fill_color='lightcyan',
               align='left'))
])
plotly.io.write_image(table, file="./images/agent_average_table.png", scale=5)
table.show()

In [6]:
agent_6_df = results_df[(results_df.Agent == 6)]
agent_7_df = results_df[(results_df.Agent == 7)]
agent_8_df = results_df[(results_df.Agent == 8)]

In [62]:
avg_results_df = avg_results_df.convert_dtypes()
bar_chart = px.bar(avg_results_df, x="Agent", y="Actions", color="Terrain", barmode="group", title="Average Actions vs Agent", text='Actions')
bar_chart.update_traces(texttemplate='%{text:.2s}', textposition='outside')
bar_chart.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
bar_chart.show()
plotly.io.write_image(bar_chart, file="./images/bar_chart_actions_terrain.png", scale=5)

In [8]:
agent_6_res = [float(r) for r in agent_6_df['Ratio']]
agent_7_res = [float(r) for r in agent_7_df['Ratio']]
agent_8_res = [float(r) for r in agent_8_df['Ratio']]

# hist_data = [agent_6_res, agent_7_res, agent_8_res]
# group_labels = ['Agent 6', 'Agent 7', 'Agent 8']
# fig_dist = ff.create_distplot(hist_data, group_labels, bin_size=.2)
# fig_dist.update_layout(title_text='Movement/Examinations Ratio Hist and Curve Plot')
# fig_dist.show()

In [9]:
box_plot = px.box(results_df, y="Ratio", color="Agent", points='all')
box_plot.update_layout(title_text='Movement/Examinations Ratio Box Plot')
box_plot.show()

In [31]:
plotly.io.write_image(fig_dist, file="./images/question_4_histogram.png", scale=5)
plotly.io.write_image(box_plot, file="./images/question_4_box.png", scale=5)

In [29]:
hist_data = [agent_6_res]
group_labels = ['Agent 6']
fig_dist = ff.create_distplot(hist_data, group_labels, bin_size=0.2, colors = ['#636EFA'])
fig_dist.update_layout(title_text='Movement/Examinations Ratio Distplot Agent 6')
fig_dist.layout.xaxis.update({'title': 'Movement/Examinations'})
fig_dist.layout.yaxis.update({'title': 'Probability Density'})
fig_dist.show()
plotly.io.write_image(fig_dist, file="./images/question_4_agent_6.png", scale=5)

In [28]:
hist_data = [agent_7_res]
group_labels = ['Agent 7']
fig_dist = ff.create_distplot(hist_data, group_labels, bin_size=0.2, colors = ['#EF553B'])
fig_dist.update_layout(title_text='Movement/Examinations Ratio Distplot Agent 7')
fig_dist.layout.xaxis.update({'title': 'Movement/Examinations'})
fig_dist.layout.yaxis.update({'title': 'Probability Density'})
fig_dist.show()
plotly.io.write_image(fig_dist, file="./images/question_4_agent_7.png", scale=5)

In [27]:
hist_data = [agent_8_res]
group_labels = ['Agent 8']
fig_dist = ff.create_distplot(hist_data, group_labels, bin_size=0.2, colors = ['#00CC96'])
fig_dist.update_layout(title_text='Movement/Examinations Ratio Distplot Agent 8')
fig_dist.layout.xaxis.update({'title': 'Movement/Examinations'})
fig_dist.layout.yaxis.update({'title': 'Probability Density'})
fig_dist.show()
plotly.io.write_image(fig_dist, file="./images/question_4_agent_8.png", scale=5)